In [1]:
from Acesso import Login
from Query import Query
from Moeda import Moeda
import pandas as pd

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

querys={

    'Meta':

    """

    SELECT * FROM netfeira.vw_metas
    WHERE [Meta R$]>0
    
    """,

    'Vendedor':

    """
    
    SELECT * FROM netfeira.vw_vendedor
    WHERE Categoria='CLT' AND [Status do Vendedor]='ATIVO' AND Telefone IS NOT NULL
    
    """,

    'Supervisor':

    """
    
    SELECT * FROM netfeira.vw_supervisor
    WHERE NOT Equipe LIKE '%120%'
    
    """,

    'Estatico':

    """
    
    DECLARE @DTBASE DATETIME, @DTFIM DATETIME,@DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Tipo de Operação]='VENDAS'
    
    """,

    'Aberto':

    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SELECT *
	FROM netfeira.vw_aberto
    WHERE [Data do Pedido]=@DTBASE
    
    """,

    'Calendario':

    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SELECT COUNT(DATA) AS [Útil],COUNT([Data Trabalhada])-1 AS [Trabalhado],
    COUNT(DATA)-(COUNT([Data Trabalhada])-1) AS [Restante]
    FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(@DTBASE) AND MONTH(Data)=MONTH(@DTBASE) AND [Dia Útil]=1    
    
    """

}

In [2]:
df=sql.CriarTabela(kwargs=querys)

C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 c

In [3]:
df['Estatico'].columns

Index(['ID Empresa', 'Pedido', 'Nfe', 'ID Cliente', 'ID Vendedor',
       'Data de Emissão', 'Data de Faturamento', 'ID Motivo', 'Situação',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Roteiro', 'ID Usuário',
       'Tabelas', 'Origem', 'Tipo de Entrega', 'Seq Roteiro', 'SKU', 'Seq',
       'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda', 'Total AV',
       'Total Geral', 'Margem Bruta R$', 'Comissão R$', 'IPI R$', 'PIS R$',
       'COFINS R$', 'ICMS R$', 'ICMS ST R$', 'Peso Bruto KG',
       'Peso Líquido KG', 'Cad Vendedor'],
      dtype='object')

In [4]:
df['Consolidado']=df['Estatico'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Consolidado'].rename(columns={'Total Venda':'Faturado'},inplace=True)

In [5]:
df['Pendente']=df['Aberto'].loc[df['Aberto']['Situação']=='AB'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Pendente'].rename(columns={'Total Venda':'Em Aberto'},inplace=True)

In [6]:
df['Meta']=df['Meta'].groupby(['ID Vendedor'],as_index=False).agg({'Meta R$':'sum'})

In [7]:
df['Meta']=df['Meta'].merge(df['Consolidado'],on='ID Vendedor',how='left')

df['Meta']=df['Meta'].merge(df['Pendente'],on='ID Vendedor',how='left')

In [8]:
colunas=df['Meta'].columns[1:]

for coluna in colunas:
    
    df['Meta'].loc[df['Meta'][coluna].isnull(),coluna]=0    
    
    pass

df['Meta']

,Meta R$,Faturado,ID Vendedor,Em Aberto


In [9]:
df['Meta']['Realizado R$']=round(df['Meta']['Faturado']+df['Meta']['Em Aberto'],2)

In [10]:
df['Meta']['Dif']=round(df['Meta']['Realizado R$']-df['Meta']['Meta R$'],2)

In [11]:
df['Meta']['Perc']=round(df['Meta']['Realizado R$']/df['Meta']['Meta R$'],4)*100

In [12]:
util=df['Calendario']['Útil'].max()

trabalhado=df['Calendario']['Trabalhado'].max()

In [13]:
df['Meta']['Projeção']=round((df['Meta']['Realizado R$']/trabalhado)*util,2)

In [14]:
df['Meta']['Perc Projeção']=round(df['Meta']['Projeção']/df['Meta']['Meta R$'],4)*100

In [15]:
colunas=df['Meta'].columns[1:]

for coluna in colunas:
    
    df['Meta'].loc[df['Meta'][coluna].isnull(),coluna]=0    
    
    pass

df['Meta']

,Meta R$,Faturado,ID Vendedor,Em Aberto,Realizado R$,Dif,Perc,Projeção,Perc Projeção


In [16]:
df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'Status do Vendedor', 'DDD',
       'Telefone'],
      dtype='object')

In [17]:
df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup', 'DDD Sup',
       'Telefone Sup', 'ID Gerente', 'Gerente', 'Email Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

In [18]:
df['Vendedor']=df['Vendedor'].merge(df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe','DDD',
       'Telefone']]

In [19]:
df['Vendedor']=df['Vendedor'].merge(df['Meta'],on='ID Vendedor',how='inner')